# 1. 상관관계

In [87]:
# 데이터
import pandas as pd
df = pd.DataFrame({
    '키': [150, 160, 170, 175, 165],
    '몸무게': [42, 52, 75, 67, 56]
})

In [88]:
# 상관계수
df.corr()

,키,몸무게
키,1.000000,0.901012
몸무게,0.901012,1.000000


In [89]:
df['키'].corr(df['몸무게'])

0.901011928876437

In [90]:
# 피어슨
print(df.corr())

# 스피어맨
print(df.corr(method='spearman'))

# 캔달타우
print(df.corr(method='kendall'))

            키       몸무게
키    1.000000  0.901012
몸무게  0.901012  1.000000
       키  몸무게
키    1.0  0.9
몸무게  0.9  1.0
       키  몸무게
키    1.0  0.8
몸무게  0.8  1.0


In [91]:
# 상관관계가 있는지에 대한 t검정
# 귀무가설: 두 변수 간에 상관관계가 없다.
# 대립가설: 두 변수 간에 상관관계가 있다.
from scipy import stats

# 피어슨
print(stats.pearsonr(df['키'], df['몸무게'])) # 귀무가설 기각

# 스피어맨
print(stats.spearmanr(df['키'], df['몸무게'])) # 귀무가설 기각

# 켄달타우 -> 출제 가능서어 낮음
print(stats.kendalltau(df['키'], df['몸무게'])) # 귀무가설 채택


PearsonRResult(statistic=0.9010119288764369, pvalue=0.03682580945087895)
SignificanceResult(statistic=0.8999999999999998, pvalue=0.03738607346849874)
SignificanceResult(statistic=0.7999999999999999, pvalue=0.08333333333333333)


# 단순 선형 회귀 분석

주어진 키와 몸무게 데이터로 회귀모델을 구축하고 각 소문제의 값을 구하시오.
- 키: 종속변수
- 몸무게: 독립변수

In [92]:
import pandas as pd

df = pd.DataFrame({
    '키': [150, 160, 170, 175, 165, 155, 172, 168, 174, 158,
          162, 173, 156, 159, 167, 163, 171, 169, 176, 161],
    '몸무게': [74, 50, 70, 64, 56, 48, 68, 60, 65, 52,
            54, 67, 49, 51, 58, 55, 69, 61, 66, 53]
})

In [93]:
# ols로 모델 학습 summary 출력
from statsmodels.formula.api import ols

model = ols('키 ~ 몸무게', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      키   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     6.984
Date:                Wed, 19 Jun 2024   Prob (F-statistic):             0.0165
Time:                        14:41:18   Log-Likelihood:                -64.701
No. Observations:                  20   AIC:                             133.4
Df Residuals:                      18   BIC:                             135.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    135.8209     11.211     12.115      0.0

In [94]:
# 결정계수
# R-squared: 0.280
model.rsquared

0.27954323113299584

In [95]:
# 독립변수에 대한 기울기(회귀계수)
# coef 0.4938
model.params['몸무게']

0.49376558603491283

In [96]:
# 절편(회귀계수)
# Intercept / coef 135.8209
model.params['Intercept']

135.8209476309227

In [97]:
# 몸무게의 회귀계수가 통계적으로 유의한지 pvalue
# 몸무게 / P>|t| 0.017
model.pvalues['몸무게']

0.01654013445316978

In [98]:
# 몸무게가 50일때 예측 키
newdata = pd.DataFrame({'몸무게':[50]})
model.predict(newdata)

0    160.509227
dtype: float64

# 잔차 제곱합
# 잔차 = 관측(실제)값 - 예측값
df['잔차'] = df['키'] - model.predict(df['몸무게'])
sum(df['잔차'] ** 2)

In [99]:
sum(model.resid ** 2)

755.9032418952607

In [100]:
# 잔차 제곱합
# 잔차 = 관측(실제)값 - 예측값

# print(model.predict(df['몸무게']).head(3))
# print(model.predict(df).head(3))

df['잔차'] = df['키'] - model.predict(df['몸무게'])
sum(df['잔차'] ** 2)

755.9032418952607

In [101]:
df['잔차']

0    -22.359601
1     -0.509227
2     -0.384539
3      7.578055
4      1.528180
5     -4.521696
6      2.602993
7      2.553117
8      6.084289
9     -3.496758
10    -0.484289
11     4.096758
12    -4.015461
13    -2.002993
14     2.540648
15     0.021945
16     1.109227
17     3.059352
18     7.590524
19    -0.990524
Name: 잔차, dtype: float64

In [102]:
# MSE: 회귀모델 평가지표
(df['잔차'] ** 2).mean()

37.79516209476303

In [103]:
# sklearn mse
from sklearn.metrics import mean_squared_error

pred = model.predict(df['몸무게'])
mean_squared_error(df['키'], pred)

37.79516209476303

In [104]:
# 신뢰구간
# 몸무게의 95% 신뢰구간을 구하시오.
#
# [0.025      0.975]
# 0.101       0.886
model.conf_int(alpha=0.05)

,0,1
Intercept,112.267778,159.374117
몸무게,0.101235,0.886297


In [105]:
# 예측값에 대한 신뢰구간과 예측구간
# ex) 몸무게가 50일 때, 예측키에 대한 신뢰구간, 예측구간

newdata = pd.DataFrame({'몸무게': [50]})
pred = model.get_prediction(newdata['몸무게'])

pred.summary_frame(alpha=0.05) # 90%면 alpha=0.1

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,160.509227,2.291332,155.695318,165.323136,146.068566,174.949888


In [106]:
# 예측값: 160.509227
# 95% 신뢰구간: 155.695318	165.323136
# 95% 예측구간: 146.068566	174.949888

# 다중 선형 회귀 분석

주어진 매출액, 광고비, 플랫폼 데이터로 회귀모델을 구축하고 각 소문제의 값을 구하시오.
- 매출액: 종속변수
- 광고비, 플랫폼(유통 플랫폼 수), 투자: 독립변수

In [107]:
import pandas as pd
df = pd.DataFrame({
    '매출액': [300, 320, 250, 360, 315, 328, 310, 335, 326, 280,
            290, 300, 315, 328, 310, 335, 300, 400, 500, 600],
    '광고비': [70, 75, 30, 80, 72, 77, 70, 82, 70, 80,
            68, 90, 72, 77, 70, 82, 40, 20, 75, 80],
    '플랫폼': [15, 16, 14, 20, 19, 17, 16, 19, 15, 20,
            14, 5, 16, 17, 16, 14, 30, 40, 10, 50],
    '투자':[100, 0, 200, 0, 10, 0, 5, 0, 20, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    })
df.head(3)

,매출액,광고비,플랫폼,투자
0,300,70,15,100
1,320,75,16,0
2,250,30,14,200


In [108]:
# 모델 학습 summary 출력
# 종속변수: 매출액
# 독립변수: 광고비, 플랫폼

from statsmodels.formula.api import ols

model = ols('매출액 ~ 광고비 + 플랫폼', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    매출액   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     8.907
Date:                Wed, 19 Jun 2024   Prob (F-statistic):            0.00226
Time:                        14:41:19   Log-Likelihood:                -108.22
No. Observations:                  20   AIC:                             222.4
Df Residuals:                      17   BIC:                             225.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    101.0239     71.716      1.409      0.1

In [109]:
# 결정계수 R-squared
# R-squared: 0.512

model.rsquared

0.5116964327009041

In [110]:
# 전체 회귀계수
model.params

Intercept    101.023872
광고비            1.819427
플랫폼            5.928756
dtype: float64

In [111]:
# 회귀계수 (광고비) : 광고비의 기울기
# 광고비 / coef : 1.8194

model.params['광고비']

1.8194269419344642

In [112]:
# 회귀계수 (플랫폼) : 플랫폼의 기울기
# 플랫폼 / coef : 5.9288

model.params['플랫폼']

5.928755546950749

In [113]:
# 회귀계수 (절편)
# Intercept / coef : 101.0239

model.params['Intercept']

101.02387228241548

In [114]:
# 광고비의 회귀계수가 통계적으로 유의한지 pvalue
# 광고비 / P>|t| : 0.038

model.pvalues['광고비'] # 귀무가설 기각

0.03764350647695994

In [115]:
# 플랫폼의 회귀계수가 통계적으로 유의한지 pvalue
# 플랫폼 / P>|t| : 0.001

model.pvalues['플랫폼'] # 귀무가설 기각

0.0006746810554564365

In [116]:
# 광고비 50, 플랫폼 20의 매출액 예측
newdata = pd.DataFrame({
    '광고비':[50],
    '플랫폼':[20]
})
model.predict(newdata)

0    310.57033
dtype: float64

In [117]:
# 잔차 제곱합
# model.resid # 잔차

print((model.resid ** 2).sum())
print(sum(model.resid ** 2))

58686.17827156107
58686.178271561075


In [118]:
# mse
(model.resid ** 2).mean()

2934.3089135780533

In [119]:
# 광고비, 플랫폼 회귀계수의 95% 신뢰구간
#       [0.025      0.975]
# 광고비  0.117       3.522
# 플랫폼  2.912       8.945

model.conf_int(alpha=0.05)

,0,1
Intercept,-50.283684,252.331429
광고비,0.116785,3.522069
플랫폼,2.912406,8.945105


In [129]:
# 광고비 50, 플랫폼 20일 때 매출액에 대한 95% 신뢰구간과 예측구간

newdata = pd.DataFrame({
    '광고비': [50],
    '플랫폼': [20]
})

pred = model.get_prediction(newdata)
pred.summary_frame(alpha=0.05)

# 신뢰구간: 268.612221	352.52844
# 예측구간: 179.700104	441.440556

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,310.57033,19.887098,268.612221,352.52844,179.700104,441.440556


### 범주형 변수

In [130]:
import pandas as pd
df = pd.DataFrame({
    '매출액': [300, 320, 250, 360, 315, 328, 310, 335, 326, 280,
            290, 300, 315, 328, 310, 335, 300, 400, 500, 600],
    '광고비': [70, 75, 30, 80, 72, 77, 70, 82, 70, 80,
            68, 90, 72, 77, 70, 82, 40, 20, 75, 80],
    '플랫폼': [15, 16, 14, 20, 19, 17, 16, 19, 15, 20,
            14, 5, 16, 17, 16, 14, 30, 40, 10, 50],
    '투자':[100, 0, 200, 0, 10, 0, 5, 0, 20, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '유형':['B','B','C','A','B','B','B','B','B','B'
        ,'C','B','B','B','B','B','B','A','A','A'] # 범주형변수
    })
df.head(3)

,매출액,광고비,플랫폼,투자,유형
0,300,70,15,100,B
1,320,75,16,0,B
2,250,30,14,200,C


In [132]:
from statsmodels.formula.api import ols

model = ols('매출액 ~ 광고비 + 유형', df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    매출액   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     13.70
Date:                Wed, 19 Jun 2024   Prob (F-statistic):           0.000110
Time:                        15:20:16   Log-Likelihood:                -102.67
No. Observations:                  20   AIC:                             213.3
Df Residuals:                      16   BIC:                             217.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    400.6463     47.477      8.439      0.0

In [134]:
# 원핫인코딩
df2 = pd.get_dummies(df) # 다중공선성 문제 발생 -> 첫번째 컬럼 drop
df2.head()

,매출액,광고비,플랫폼,투자,유형_A,유형_B,유형_C
0,300,70,15,100,False,True,False
1,320,75,16,0,False,True,False
2,250,30,14,200,False,False,True
3,360,80,20,0,True,False,False
4,315,72,19,10,False,True,False


In [135]:
# 다중공선성 문제 발생 -> 첫번째 컬럼 drop
# drop_first = True
# B, C 둘 다 False, 0이면 A

df = pd.get_dummies(df, drop_first=True)
df.head()

,매출액,광고비,플랫폼,투자,유형_B,유형_C
0,300,70,15,100,True,False
1,320,75,16,0,True,False
2,250,30,14,200,False,True
3,360,80,20,0,False,False
4,315,72,19,10,True,False


In [137]:
# 회귀 모델 학습 & summary (독립변수로 광고비와 유형만 활용)
from statsmodels.formula.api import ols

model = ols('매출액 ~ 광고비 + 유형_B + 유형_C', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    매출액   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     13.70
Date:                Wed, 19 Jun 2024   Prob (F-statistic):           0.000110
Time:                        15:25:29   Log-Likelihood:                -102.67
No. Observations:                  20   AIC:                             213.3
Df Residuals:                      16   BIC:                             217.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      400.6463     47.477      8.439   

In [144]:
# 결정계수
model.rsquared

0.7197356505841288

In [152]:
model.pvalues

Intercept       2.755012e-07
유형_B[T.True]    1.887812e-05
유형_C[T.True]    4.421253e-04
광고비             1.411180e-01
dtype: float64

In [147]:
'{:.11f}'.format(2.755012e-07)

'0.00000027550'

In [153]:
for pvalue in model.pvalues:
    print('{:.11f}'.format(pvalue))

0.00000027550
0.00001887812
0.00044212527
0.14111798846


In [159]:
# mse
 (model.resid ** 2).mean()

1684.1617258663587

In [161]:
# mse
from sklearn.metrics import mean_squared_error
pred = model.predict(df)
mean_squared_error(df['매출액'], pred)

1684.1617258663587

In [140]:
model.params # 회귀계수

Intercept       400.646316
유형_B[T.True]   -160.269547
유형_C[T.True]   -180.110324
광고비               1.009470
dtype: float64

In [165]:
model.conf_int(alpha=0.05) # 95% 신뢰구간

,0,1
Intercept,300.000282,501.292351
유형_B[T.True],-216.990779,-103.548315
유형_C[T.True],-266.777715,-93.442933
광고비,-0.372742,2.391681
